In [13]:
import pandas as pd
import numpy as np

#Import the original file
orig_df = pd.read_csv('../Movies/Data/movies_metadata.csv', low_memory=False)

#import data from the clean file
df = pd.read_csv('../Movies/Data/metadata_clean.csv')

#Add the useful features into the cleaned dataframe
df['overview'], df['id'] = orig_df['overview'], orig_df['id']

#print the head of the cleaned Dataframe
df.head()


,title,genres,runtime,vote_average,vote_count,year,overview,id
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862


In [14]:
# Load the keywords and credits files
cred_df = pd.read_csv('../Movies/the-movies-dataset/credits.csv')
key_df = pd.read_csv('../Movies/the-movies-dataset/keywords.csv')

#Print the head of the credit dataframe
cred_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [18]:
# convert the IDs of df int
df['id'] = df['id'].astype('int')

ValueError: invalid literal for int() with base 10: '1997-08-20'

In [20]:
# remove basd IDs 
# function to convert all non-integer IDs to NaN
def clean_ids(x):
    try:
        return int(x)
    except:
        return np.nan

In [23]:
# clean the ids of df
df['id'] = df['id'].apply(clean_ids)

#filter all row that have a null ID
df = df[df['id'].notnull()]

In [25]:
# we are now good pos to  convert the IDs of all three dataframes into integers & merge them

# convert IDs into integer
df['id'] = df['id'].astype('int')
key_df['id'] = key_df['id'].astype('int')
cred_df['id'] = cred_df['id'].astype('int')


# merge keywords & credits into your main metadata dataframe
df = df.merge(cred_df, on='id')
df = df.merge(key_df, on='id')


#Display the head of the merged df
df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id,cast,crew,keywords
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


##### wrangling keyword, cast & crew
<li>Convert "keywords" into a list of strings where each string is a keyword (similar to genres). We will include only the top three keywords. Therefore, this list can have a maximum of three elements.</li>
<li>Convert "cast" into a list of strings where each string is a star. 
Like "keywords", we will only include the top three stars in our cast. </li>
<li>Convert "crew" into director. In other words, we will extract only the director of the movie and ignore all other crew members.</li>

In [26]:
# convert the stringified objects into the native python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    df[feature] = df[feature].apply(literal_eval)
    

In [36]:
#print the first cast member fo the first movie in df
df.iloc[0]['crew'][0]

{'credit_id': '52fe4284c3a36847f8024f49',
 'department': 'Directing',
 'gender': 2,
 'id': 7879,
 'job': 'Director',
 'name': 'John Lasseter',
 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

In [37]:
# Extract the director's name. if director is not listed, return NaN
def get_director(x):
    for crew_member in x:
        if crew_member['job'] == 'Director':
            return crew_member['name']
    return np.nan

In [38]:
# Define the new director feature
df['director'] = df['crew'].apply(get_director)

# print the directores of the first five movies
df['director'].head()

0      John Lasseter
1       Joe Johnston
2      Howard Deutch
3    Forest Whitaker
4      Charles Shyer
Name: director, dtype: object

In [48]:
# Return the list top 3 elements or entire list, whichever is more:
def generate_list(x):
    if isinstance(x, list):
        names = [ele['name'] for ele in x]
        # check if more than 3 elements exist.if yes, return only first three
        # if no, return entire list
        if len(names) > 3 :
            names = names[:3]
        return names
    
    # return empty list in case of missing/malformed data
    return []

In [49]:
# Apply the generate_list function to cast & keywords
df['cast'] = df['cast'].apply(generate_list)
df['keywords'] = df['keywords'].apply(generate_list)

# only consider a max of 3 genres
df['genres'] = df['genres'].apply(lambda x: x[:3])

In [50]:
# print the new features of the first 5 movies along the title
df[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[animation, comedy, family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[adventure, fantasy, family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[romance, comedy]"


In [60]:
# Function to sanitize data to prevent ambiquity
# Remove spaces & converts to lowercase

def sanitize(x):
    if isinstance(x, list):
        # strip spaces & convert to lowercase
        return [str.lower(i.replace(' ', '')) for i in x]
    else:
        # check if director exists. if not return empty str
        if isinstance(x, str):
            return str.lower(x.replace(' ', ''))
        else:
            return ''

In [84]:
# Apply the generate_list func to cast, keywords, director & genres
for features in ['cast', 'director', 'genres', 'keywords']:
    df[features] = df[features].apply(sanitize)

In [86]:
# creating the metadata soup

#Function that creates a soup out of the desired metadata
def create_soup(x):
        return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])


In [87]:
# create the new soup feature
df['soup'] = df.apply(create_soup, axis=1)

In [88]:
# Display the soup of the first movie
df.iloc[0]

title                                                   Toy Story
genres                                [animation, comedy, family]
runtime                                                        81
vote_average                                                  7.7
vote_count                                                   5415
year                                                         1995
overview        Led by Woody, Andy's toys live happily in his ...
id                                                            862
cast                             [tomhanks, timallen, donrickles]
crew            [{'credit_id': '52fe4284c3a36847f8024f49', 'de...
keywords                                     [jealousy, toy, boy]
director                                             johnlasseter
soup            jealousy toy boy tomhanks timallen donrickles ...
Name: 0, dtype: object

#### Generating the recommendation

In [90]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Define new counterVectorizer obj & create vectors for the soup
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['soup'])

In [95]:
# import cosine_similarity function
from sklearn.metrics.pairwise import cosine_similarity

# compute the cosine similarity score(equiv to dot product for tf-idf vectors)
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# Reset index of our df & construct reverse mapping again
df = df.reset_index()
indices2 = pd.Series(df.index, index=df['title'])

In [ ]:
content_recommender('The Lion King', cosine_sim2, df, indices2)